In [1]:
#import mariadb
import tqdm
import pickle
import pandas as pd
import mysql.connector
from decimal import Decimal
from neo4j import GraphDatabase
from datetime import timedelta

In [2]:
df_pw = pd.read_csv("pw.csv", delimiter=",")

In [3]:
df_sqlcsv = pd.read_csv('reviews_price_btc.csv')
df_sqlcsv.time = pd.to_datetime(df_sqlcsv.time)
df_sqlcsv["price_btc_reviews_satoshi"] = df_sqlcsv["price_btc_review"] * 100000000
df_sqlcsv = df_sqlcsv.sort_values(by="time",ascending=True)
df_sqlcsv.head(3)

,Unnamed: 0,price_eur,time,customer,vendor_id,vendor_review_id,price_btc_review,price_btc_reviews_satoshi
28037,28037,29.27,2019-07-29 15:49:29,n***0 [3],b7906806e9f4a5148a23b66f6c0ac0ed,106810,0.003435,343484.128381
18100,18100,72.25,2019-07-29 15:11:53,k***s,7ae5fd6d6e8d45455fd29d0c7fd6b3f7,26604,0.008479,847855.424514
18818,18818,60.24,2019-07-29 15:10:23,b***6,a86eaa443894c015eb7ff280d7597ed7,27652,0.007069,706917.796163


In [4]:
db = GraphDatabase.driver("neo4j://127.0.0.1:7687", auth=(df_pw["name"][0], df_pw["password"][0]))
session = db.session()

In [5]:
df_neo4jbig = pd.DataFrame(columns=['date', 'inDegree', 'inSum', 'outDegree', 'outSum', 'txid','vendor'])
df_neo4jbiga = pd.DataFrame(columns=['inDegree', 'outDegree', 'address','vendor'])
for i in df_sqlcsv.itertuples():
    neo_results = []
    neo_results2 = []
    start = str(i.time.date()- timedelta(days=3))
    end = str(i.time.date())
    minSum = str(round(i.price_btc_reviews_satoshi,0))
    maxSum = str(round(i.price_btc_reviews_satoshi * 1.05,0))
    neo_query = session.run("""
    WITH '""" + start + """' as minDate,  '""" + end + """' as maxDate, """
    + minSum + """ as minSum, """ + maxSum + """ as maxSum
    MATCH (t:Transaction) 
        WHERE  t.inDegree = 1 and ( t.outDegree = 2 OR t.outDegree = 1 ) AND date(minDate) <= t.date <= date(maxDate) 
        and minSum <= t.inSum <= maxSum
    WITH t, minSum, minDate, maxDate
    MATCH (t)-[r:RECEIVES]->(a:Address),
        (t)-[r1:RECEIVES]->(c:Address)-[s:SENDS]->(t)
        WHERE minSum  <= r.value <= minSum * 1.02 AND c <> a
    WITH a, r, minDate, maxDate
    MATCH (a)-[s:SENDS]->(mix:Transaction)
        WHERE date(minDate) <= mix.date <= date(maxDate) AND r.value >= s.value 
    RETURN mix, a
    """)
    for r in neo_query:
        neo_results.append(r[0])
        neo_results2.append(r[1])
    df_neo4j = pd.DataFrame(neo_results)
    df_neo4j["vendor"] = i.vendor_id
    df_neo4j["review"] = i.vendor_review_id
    df_neo4jbig = pd.concat([df_neo4jbig,df_neo4j])
    
    df_neo4j2 = pd.DataFrame(neo_results2)
    df_neo4j2["vendor"] = i.vendor_id
    df_neo4j2["review"] = i.vendor_review_id
    df_neo4jbiga = pd.concat([df_neo4jbiga,df_neo4j2])
    
display(df_neo4jbig.head() )
display(df_neo4jbiga.head() )

,date,inDegree,inSum,outDegree,outSum,txid,vendor,review
0,2019-07-27,8,2834572,1,2831588.0,3ea0e13f1740f437189f8bc80a3d713ab2f07946525e6c...,b7906806e9f4a5148a23b66f6c0ac0ed,106810.0
1,2019-07-29,51,101843003,0,NaN,414a4612d5eb8df6d798a1ae969d036d65eb6245b9bc10...,b7906806e9f4a5148a23b66f6c0ac0ed,106810.0


,inDegree,outDegree,address,vendor,review
0,1,1,1LHMKzHsmKrsU6BzX7bTkRLXr768KNZ7he,b7906806e9f4a5148a23b66f6c0ac0ed,106810.0
1,1,1,3GuLymm9gigDk7m2B3qK9j8qyaYBXy4WaE,b7906806e9f4a5148a23b66f6c0ac0ed,106810.0


In [6]:
df_neo4jbig.to_csv("QueryResultsTransaction1Mix.csv")
df_neo4jbiga.to_csv("QueryResultsTransaction1Adr.csv")

In [7]:
session.close()